# Tugas Case Based Learning
Menghitung resiko tingkat terkena penyakit bagi orang yang merokok, dengan menggunakan teknik learning Decision Tree.





# 📊 Preprocessing Data
---
Tahap awal yang bertujuan untuk menyiapkan dan membersihkan data mentah agar siap digunakan oleh algoritma decision tree.

In [ ]:
import pandas as pd

In [ ]:
# Baca data dari file csv
url = "https://raw.githubusercontent.com/irfanrangga/smoker-health-risk-prediction-ML/refs/heads/main/smoking_health_data_final.csv"
df = pd.read_csv(url)

# cetak 5 data pertama
df.head()

,age,sex,current_smoker,heart_rate,blood_pressure,cigs_per_day,chol
0,54,male,yes,95,110/72,NaN,219.0
1,45,male,yes,64,121/72,NaN,248.0
2,58,male,yes,81,127.5/76,NaN,235.0
3,42,male,yes,90,122.5/80,NaN,225.0
4,42,male,yes,62,119/80,NaN,226.0


## Kategorisasi Data
*   Mengubah Data Kontinu menjadi Data Diskrit
*   Mengisi data kosong dengan rata-rata nilai
*   Membagi kolom 'blood_presure' menjadi 2 untuk mendapatkan nilai Sistol dan Diastol yang akurat



In [ ]:
# Membagi kolom blood_pressure menjadi 2
df[['bp_sys', 'bp_dia']] = df['blood_pressure'].str.split('/', expand=True)
df['bp_sys'] = pd.to_numeric(df['bp_sys'])
df['bp_dia'] = pd.to_numeric(df['bp_dia'])

df.drop(columns=['blood_pressure'], inplace=True)

# Data Diskrit Umur
df['age_cat'] = pd.cut(df['age'],
                       bins = [0, 30, 45, 60, 100],
                       labels = ['muda', 'dewasa', 'paruh_baya', 'lansia'])

# Data Diskrit Detak Jantung
df['heart_rate_cat'] = pd.cut(df['heart_rate'],
                              bins = [0, 50, 80, 150],
                              labels = ['rendah', 'normal', 'tinggi'])

# Data Diskrit Kolestrol
df['cholesterol_cat'] = pd.cut(df['chol'],
                               bins = [0, 200, 240, 1000],
                               labels = ['normal', 'sedang', 'tinggi'])

# Data Diskrit Konsumsi Rokok per hari
avg_cigs = df['cigs_per_day'].mean()
df['cigs_cat'] = pd.cut(df['cigs_per_day'].fillna(avg_cigs),
                        bins = [-1, 0, 10, 20, 100],
                        labels = ['tidak_merokok', 'ringan', 'sedang', 'berat'])

# Data Diskrit Tekanan Darah (Sistolic)
df['bp_sys_cat'] = pd.cut(df['bp_sys'],
                          bins = [0, 120, 130, 140, 180, 300],
                          labels = ['normal', 'elevated', 'hypertensi_1', 'hypertensi_2', 'krisis'])

# Data Diskrit Tekanan Darah (Diastolic)
df['bp_dia_cat'] = pd.cut(df['bp_dia'],
                          bins = [0, 80, 90, 120, 150],
                          labels = ['normal', 'hypertensi_1', 'hypertensi_2', 'krisis'])

# Cetak data setelah perubahan
df.head()

,age,sex,current_smoker,heart_rate,cigs_per_day,chol,bp_sys,bp_dia,age_cat,heart_rate_cat,cholesterol_cat,cigs_cat,bp_sys_cat,bp_dia_cat
0,54,male,yes,95,NaN,219.0,110.0,72.0,paruh_baya,tinggi,sedang,ringan,normal,normal
1,45,male,yes,64,NaN,248.0,121.0,72.0,dewasa,normal,tinggi,ringan,elevated,normal
2,58,male,yes,81,NaN,235.0,127.5,76.0,paruh_baya,tinggi,sedang,ringan,elevated,normal
3,42,male,yes,90,NaN,225.0,122.5,80.0,dewasa,tinggi,sedang,ringan,elevated,normal
4,42,male,yes,62,NaN,226.0,119.0,80.0,dewasa,normal,sedang,ringan,normal,normal


## Menentukan Fitur dan Target


### Fitur yang digunakan pada perhitungan Decision Tree ini antara lain adalah:

*   Jenis Kelamin
*   Perokok Aktif atau Tidak
*   Kategori Usia
*   Detak Jantung
*   Tingkat Kolestrol
*   Kategori Perokok
*   Tingkat Tekanan Darah (Sistolik dan Diastolik)

### Serta Target yang diharapkan adalah:

*  Mengatahui Tingkat Resiko Terkena Penyakit







In [ ]:
# Menghitung label target berdasarkan data
def hitung_risiko(row):
    skor = 0
    if row['bp_sys_cat'] == 'tinggi':
        skor += 1
    if row['cholesterol_cat'] == 'tinggi':
        skor += 1
    if row['heart_rate_cat'] == 'tinggi':
        skor += 1
    if row['cigs_cat'] in ['sedang', 'berat']:
        skor += 1
    if skor >= 2:
        return 'tinggi'
    elif skor == 1:
        return 'sedang'
    else:
        return 'rendah'

df['risiko'] = df.apply(hitung_risiko, axis=1)

# Hapus data kosong
df_cat = df[['sex', 'current_smoker', 'age_cat', 'heart_rate_cat', 'bp_sys_cat', 'cholesterol_cat', 'cigs_cat', 'risiko']].dropna()

# Fitur Kategori dan Target yang ingin dicapai
fitur_kategorikal = ['sex', 'current_smoker', 'age_cat', 'heart_rate_cat', 'bp_sys_cat', 'cholesterol_cat', 'cigs_cat']
target = 'risiko'

df_cat.head(10)

,sex,current_smoker,age_cat,heart_rate_cat,bp_sys_cat,cholesterol_cat,cigs_cat,risiko
0,male,yes,paruh_baya,tinggi,normal,sedang,ringan,sedang
1,male,yes,dewasa,normal,elevated,tinggi,ringan,sedang
2,male,yes,paruh_baya,tinggi,elevated,sedang,ringan,sedang
3,male,yes,dewasa,tinggi,elevated,sedang,ringan,sedang
4,male,yes,dewasa,normal,normal,sedang,ringan,rendah
5,male,yes,paruh_baya,normal,normal,sedang,ringan,rendah
6,male,yes,dewasa,normal,normal,sedang,ringan,rendah
7,male,yes,dewasa,normal,elevated,normal,ringan,rendah
8,male,yes,dewasa,normal,elevated,normal,ringan,rendah
9,male,yes,paruh_baya,tinggi,elevated,tinggi,ringan,tinggi


# Implementasi Algoritma Iterative Dichotomiser Tree (ID3)

In [ ]:
import math
from collections import Counter

# 1. Entropi
def entropy(labels):
  total = len(labels)
  counts = Counter(labels)
  return -sum([count / total * math.log(count / total, 2) for count in counts.values()])

# 2. Information Gain
def info_gain(data, feature, target):
    total_entropy = entropy(data[target])
    values = data[feature].unique()
    weighted_entropy = 0
    for val in values:
        subset = data[data[feature] == val]
        weighted_entropy += (len(subset)/len(data)) * entropy(subset[target])
    return total_entropy - weighted_entropy

# 3. ID3 Recursive
def id3(data, features, target):
    labels = list(data[target])
    if labels.count(labels[0]) == len(labels):
        return labels[0]
    if len(features) == 0:
        return Counter(labels).most_common(1)[0][0]

    gains = [(feat, info_gain(data, feat, target)) for feat in features]
    best_feat = max(gains, key=lambda x: x[1])[0]

    tree = {best_feat: {}}
    for val in data[best_feat].unique():
        subset = data[data[best_feat] == val]
        subtree = id3(subset, [f for f in features if f != best_feat], target)
        tree[best_feat][val] = subtree

    return tree

In [ ]:
import pprint

features = ['sex', 'current_smoker', 'age_cat', 'heart_rate_cat', 'cholesterol_cat', 'bp_sys_cat', 'cigs_cat']
target = 'risiko'

tree = id3(df_cat, features, target)

pprint.pprint(tree)

{'cholesterol_cat': {'normal': {'cigs_cat': {'berat': {'heart_rate_cat': {'normal': 'sedang',
                                                                          'rendah': 'sedang',
                                                                          'tinggi': 'tinggi'}},
                                             'ringan': {'heart_rate_cat': {'normal': 'rendah',
                                                                           'rendah': 'rendah',
                                                                           'tinggi': 'sedang'}},
                                             'sedang': {'heart_rate_cat': {'normal': 'sedang',
                                                                           'rendah': 'sedang',
                                                                           'tinggi': 'tinggi'}},
                                             'tidak_merokok': {'heart_rate_cat': {'normal': 'rendah',
                                        

# Evaluasi Hasil

In [ ]:
def predict(tree, sample):
    if not isinstance(tree, dict):
        return tree  # jika sudah mencapai daun, return label

    # Ambil nama fitur pertama dari pohon (akar sekarang)
    feature = next(iter(tree))

    # Ambil nilai dari fitur di sampel
    feature_value = sample.get(feature)

    # Cek apakah nilai fitur ini tersedia di cabang pohon
    subtree = tree[feature].get(feature_value)

    if subtree is None:
        return "tidak diketahui"  # fallback jika nilai tidak ada di pohon

    return predict(subtree, sample)

In [ ]:
sample = {
    'sex': 'male',
    'current_smoker': 'yes',
    'age_cat': 'lansia',
    'heart_rate_cat': 'tinggi',
    'bp_sys_cat': 'normal',
    'cholesterol_cat': 'normal',
    'cigs_cat': 'berat'
}

# Aplikasikan fungsi predict pada seluruh baris data
def predict_df(tree, df):
    return df.apply(lambda row: predict(tree, row.to_dict()), axis=1)

df_cat['prediksi'] = predict_df(tree, df_cat[features])

hasil = predict(tree, sample)
print("Prediksi risiko terkena penyakit:", hasil)

akurasi = (df_cat['prediksi'] == df_cat['risiko']).mean()
print(f"Akurasi prediksi: {akurasi:.2%}")

df_cat.head()

Prediksi risiko terkena penyakit: tinggi
Akurasi prediksi: 99.97%


,sex,current_smoker,age_cat,heart_rate_cat,bp_sys_cat,cholesterol_cat,cigs_cat,risiko,prediksi
0,male,yes,paruh_baya,tinggi,normal,sedang,ringan,sedang,sedang
1,male,yes,dewasa,normal,elevated,tinggi,ringan,sedang,sedang
2,male,yes,paruh_baya,tinggi,elevated,sedang,ringan,sedang,sedang
3,male,yes,dewasa,tinggi,elevated,sedang,ringan,sedang,sedang
4,male,yes,dewasa,normal,normal,sedang,ringan,rendah,rendah
